In [ ]:
import os
import random

import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [ ]:
# Set the seed
torch.manual_seed(42)

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

device

In [ ]:
LABEL_MAP = ["upper_body", "lower_body", "dresses"]


class FashionDataset(Dataset):
    def __init__(self, root: str, pairs: str) -> None:
        super().__init__()

        self.transforms = transforms.Compose(
            [transforms.Resize((256, 192)), transforms.ToTensor()]
        )

        # Root directory of the dataset
        self.root = root

        # Load in the paired data
        self.data = pd.read_csv(
            pairs, delimiter="\t", header=None, names=["model", "garment", "label"]
        )

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> dict:
        model, garment, label = self.data.iloc[index]

        # Load the images
        anchor = Image.open(
            os.path.join(self.root, LABEL_MAP[label], "images", garment)
        ).convert("RGB")

        positive = Image.open(
            os.path.join(self.root, LABEL_MAP[label], "images", model)
        ).convert("RGB")

        # Load the negative image
        negative_index = random.randrange(0, len(self.data))
        negative_model, negative_garment, negative_label = self.data.iloc[
            negative_index
        ]

        negative = Image.open(
            os.path.join(self.root, LABEL_MAP[negative_label], "images", negative_model)
        ).convert("RGB")

        # Resize & convert to tensors
        anchor = self.transforms(anchor)
        positive = self.transforms(positive)
        negative = self.transforms(negative)

        return anchor, positive, negative

In [ ]:
def train(
    model: nn.Module,
    train_data: DataLoader,
    test_data: DataLoader,
    loss_fcn: nn.Module,
    epochs: int = 10,
    device: str = "cpu",
    log_dir: str = "./logs",
    output_dir: str = "./models",
    model_name: str = "ResNet50",
):
    # Create the log & output directories if they don't exist
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Tensorboard logger
    logger = SummaryWriter(os.path.join(log_dir, model_name))

    for epoch in range(epochs):

        # Set model to training mode
        model.train()

        for i, (anchor, positive, negative) in tqdm(
            enumerate(train_data),
            f"Epoch {epoch} Training",
            unit="batch",
            total=len(train_data),
        ):
            optimizer.zero_grad()

            # Send images to the device
            anchor = anchor.to(device)
            positive = positive.to(device)
            negative = negative.to(device)

            # Forward pass
            anchor_features = model(anchor)
            positive_features = model(positive)
            negative_features = model(negative)

            # Compute the loss
            loss = loss_fcn(anchor_features, positive_features, negative_features)

            # Log loss to tensorboard
            logger.add_scalar("Train/Triplet Loss", loss, i + epoch * len(train_data))

            # Backward pass
            loss.backward()
            optimizer.step()

        # Evaluate the model on the testing data
        model.eval()

        validation_loss = 0.0
        euclidean_distance_ap = 0.0
        euclidean_distance_an = 0.0

        with torch.no_grad():
            for i, (anchor, positive, negative) in tqdm(
                enumerate(test_data),
                f"Epoch {epoch} Evaluation",
                unit="batch",
                total=len(test_data),
            ):
                # Send images to the device
                anchor = anchor.to(device)
                positive = positive.to(device)
                negative = negative.to(device)

                # Forward pass
                anchor_features = model(anchor)
                positive_features = model(positive)
                negative_features = model(negative)

                # Compute the loss
                validation_loss += loss_fcn(
                    anchor_features, positive_features, negative_features
                )

                # Compute the Euclidean distance between anchor and positive features
                euclidean_distance_ap += torch.norm(
                    anchor_features - positive_features, dim=1
                ).sum()

                # Compute the Euclidean distance between anchor and negative features
                euclidean_distance_an += torch.norm(
                    anchor_features - negative_features, dim=1
                ).sum()

        logger.add_scalar("Test/Triplet Loss", validation_loss / len(test_data), epoch)
        logger.add_scalar(
            "Test/Euclidean Distance Ratio (AN/AP)",
            euclidean_distance_an / euclidean_distance_ap,
            epoch,
        )

        print(f"Epoch {epoch} Validation Loss: {validation_loss / len(test_data)}")

        # Save the model
        torch.save(model.state_dict(), os.path.join(output_dir, f"{model_name}.pth"))

In [ ]:
# Load the dataset
train_data = FashionDataset("data/DressCode", "data/DressCode/train_pairs.txt")

test_data = FashionDataset("data/DressCode", "data/DressCode/test_pairs_paired.txt")


# Define the training dataloader
train_loader = DataLoader(train_data, batch_size=48, shuffle=True)

# Define the validation dataloader
test_loader = DataLoader(test_data, batch_size=48, shuffle=False)

In [ ]:
# Load the model
model = models.resnet50()

model = model.to(device)

In [ ]:
train(
    model, train_loader, test_loader, nn.TripletMarginLoss(), epochs=10, device=device
)